In [16]:
import torch 
import torch.nn as nn
import torch.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets
import numpy as np
import torch.optim as optim

In [17]:
num_epochs=6
num_classes=10
batch_size=100
lr=0.001

In [12]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.layer1=nn.Sequential( nn.Conv2d(1,32,kernel_size=5,stride=1,padding=2),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2,stride=2)
                                )
        self.layer2=nn.Sequential( nn.Conv2d(32,64,kernel_size=5,stride=1,padding=2),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2,stride=2)
                                 )
        self.dropout=nn.Dropout()
        self.fc1=nn.Linear(7*7*64,1000)
        self.fc2=nn.Linear(1000,10)
    
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.reshape(out.size(0),-1)
        out=self.fc1(out)
        out=self.fc2(out)
        return out
        



In [14]:
#Load the dataset 
#The dataset is inbuilt at torchvision package
DATA_PATH="/home/iyer/MNIST_DATA"
MODEL_PATH="/home/iyer/MODEL"

transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])
train_data=torchvision.datasets.MNIST(root=DATA_PATH,train=True,transform=transform,download=True)
test_data=torchvision.datasets.MNIST(root=DATA_PATH,train=False,transform=transform)




0it [00:00, ?it/s]


 18%|█▊        | 1761280/9912422 [00:34<02:46, 49003.41it/s]


 46%|████▌     | 4521984/9912422 [01:16<01:25, 63336.46it/s]


 68%|██████▊   | 6782976/9912422 [01:48<01:05, 48115.99it/s]


 93%|█████████▎| 9256960/9912422 [02:34<00:25, 25857.03it/s]


...



  0%|          | 0/1648877 [00:00<?, ?it/s]

  1%|▏         | 24576/1648877 [00:00<00:06, 233541.66it/s]

  2%|▏         | 32768/1648877 [00:01<00:21, 74747.46it/s] 

  3%|▎         | 57344/1648877 [00:01<00:18, 87273.14it/s]

  4%|▍         | 73728/1648877 [00:01<00:17, 90417.56it/s]

  5%|▌         | 90112/1648877 [00:01<00:19, 79068.00it/s]

  6%|▌         | 98304/1648877 [00:01<00:20, 77429.34it/s]

  7%|▋         | 114688/1648877 [00:01<00:18, 81579.91it/s]

  8%|▊         | 131072/1648877 [00:02<00:16, 92920.36it/s]

  9%|▉         | 147456/1648877 [00:02<00:21, 70692.39it/s]

 10%|▉         | 163840/1648877 [00:02<00:18, 79808.76it/s]

 11%|█         | 180224/1648877 [00:03<00:26, 56133.79it/s]

 15%|█▍        | 245760/1648877 [00:03<00:18, 77060.68it/s]

 16%|█▋        | 270336/1648877 [00:03<00:15, 89407.78it/s]

 18%|█▊        | 294912/1648877 [00:03<00:18, 71594.17it/s]

 20%|█▉        | 327680/1648877 [00:03<00:14, 91840.92it/s]

 21%|██▏       | 352256/1648877 [00:04<00:

Extracting /home/iyer/MNIST_DATA/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/iyer/MNIST_DATA/MNIST/raw





  0%|          | 0/4542 [00:00<?, ?it/s]


8192it [00:00, 12570.92it/s]            


Extracting /home/iyer/MNIST_DATA/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/iyer/MNIST_DATA/MNIST/raw
Processing...
Done!




1654784it [00:31, 60835.15it/s]

In [15]:
trainloader=DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
testloader=DataLoader(dataset=test_data,batch_size=batch_size,shuffle=False)


In [23]:
model=ConvNet()

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=lr)
total_step=len(trainloader)
loss_list=[]
accs_list=[]
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(trainloader):
        output=model(images)
        loss=criterion(output,labels)
        loss_list.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #Track the Accuracy
        total=labels.size(0)
        _,prediction=torch.max(output.data,1)
        cnt=(prediction==labels).sum().item()
        accs_list.append(cnt/total)
        
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (cnt / total) * 100))
        

Epoch [1/6], Step [100/600], Loss: 0.1122, Accuracy: 94.00%
Epoch [1/6], Step [200/600], Loss: 0.0782, Accuracy: 96.00%
Epoch [1/6], Step [300/600], Loss: 0.0525, Accuracy: 99.00%
Epoch [1/6], Step [400/600], Loss: 0.0676, Accuracy: 97.00%
Epoch [1/6], Step [500/600], Loss: 0.0392, Accuracy: 99.00%
Epoch [1/6], Step [600/600], Loss: 0.0500, Accuracy: 98.00%
Epoch [2/6], Step [100/600], Loss: 0.0229, Accuracy: 98.00%
Epoch [2/6], Step [200/600], Loss: 0.0136, Accuracy: 100.00%
Epoch [2/6], Step [300/600], Loss: 0.0170, Accuracy: 100.00%
Epoch [2/6], Step [400/600], Loss: 0.0654, Accuracy: 98.00%
Epoch [2/6], Step [500/600], Loss: 0.0904, Accuracy: 98.00%
Epoch [2/6], Step [600/600], Loss: 0.1368, Accuracy: 97.00%
Epoch [3/6], Step [100/600], Loss: 0.0078, Accuracy: 100.00%
Epoch [3/6], Step [200/600], Loss: 0.0025, Accuracy: 100.00%
Epoch [3/6], Step [300/600], Loss: 0.0232, Accuracy: 98.00%
Epoch [3/6], Step [400/600], Loss: 0.0336, Accuracy: 98.00%
Epoch [3/6], Step [500/600], Loss: 0

In [25]:
model.eval()
with torch.no_grad():
    cnt=0
    total=0
    for images,labels in testloader:
        output=model(images)
        total+=labels.size(0)
        _,prediction=torch.max(output.data,1)
        cnt+=(prediction==labels).sum().item()
    print('Test Accuracy of the model on the 10000 test images: {} %'.format((cnt/ total) * 100))


Test Accuracy of the model on the 10000 test images: 98.98 %


In [26]:
torch.save(model.state_dict(), MODEL_PATH + 'conv_net_model.ckpt')

In [29]:
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d
p = figure(y_axis_label='Loss', width=850, y_range=(0, 1), title='PyTorch ConvNet results')
p.extra_y_ranges = {'Accuracy': Range1d(start=0, end=100)}
p.add_layout(LinearAxis(y_range_name='Accuracy', axis_label='Accuracy (%)'), 'right')
p.line(np.arange(len(loss_list)), loss_list)
p.line(np.arange(len(loss_list)), np.array(accs_list) * 100, y_range_name='Accuracy', color='red')
show(p)

W-1005 (SNAPPED_TOOLBAR_ANNOTATIONS): Snapped toolbars and annotations on the same side MAY overlap visually: Figure(id='d41daabc-f352-42ac-a6fe-55ce254e2caa', ...)
